In [1]:
import permutationTest as pt 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import time
import pickle

ModuleNotFoundError: No module named 'permutationTest'

In [2]:
sampleShape = [10,50,100,150,200,250,300,400,500]
n_samples = 1
bins =  500
mean =0.2
std = 1

data_variables = (sampleShape, n_samples, bins, mean, std)

In [3]:
data_generator = lambda mean, std, size ,n_samples : [np.random.normal(mean, std, size) for _ in range(n_samples)]


In [4]:
def runExperiment(method_list, names_list, data_generator, data_variables):
    sampleShape, n_samples, bins, mean, std = data_variables
    
    method_dict = dict()
    
    for method, name in zip(method_list, names_list):
        
        method_dict[name] = dict()
        method_dict[name]["p_values"] = list()
        method_dict[name]["time"] = list()
        
        for size in sampleShape:
            
            np.random.seed(42)
            A = np.asarray(data_generator(mean, std, size ,n_samples))
            B = np.asarray(data_generator(mean, std, size ,n_samples))
        
            start = time.time()
            method_dict[name]["p_values"].append(method(A,B,bins)[0])
            end = time.time()
            method_dict[name]["time"].append(end - start)
            
    return method_dict
    

In [5]:
method_list = [pt.GreenFloatCuda]
names_list = ["RTX2060"]
RTX2060_dict = runExperiment(method_list, names_list, data_generator, data_variables)

In [13]:
pickle.dump(RTX2060_dict,open("/home/ekvall/parallelGreen/notebooks/data/experiment2/RTX2060_dict.pkl", "wb"))